In [1]:
import numpy as np
import tensorflow as tf

n_steps = 4
n_inputs = 3
n_neurons = 5

In [2]:
n_steps = 2
n_inputs = 3
n_neurons = 5

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
X_seqs = tf.unstack(tf.transpose(X, perm=[1, 0, 2]))

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, X_seqs, dtype=tf.float32)
outputs = tf.transpose(tf.stack(output_seqs), perm=[1, 0, 2])

init = tf.global_variables_initializer()

In [5]:
X_batch = np.array([
        # t = 0      t = 1 
        [[0, 1, 2], [9, 8, 7]], # instance 1
        [[3, 4, 5], [0, 0, 0]], # instance 2
        [[6, 7, 8], [6, 5, 4]], # instance 3
        [[9, 0, 1], [3, 2, 1]], # instance 4
    ])

with tf.Session() as sess:
    init.run()
    print(sess.run([X_seqs], feed_dict={X: X_batch}))
    #print(sess.run(output_seqs, feed_dict={X: X_batch}))
    outputs_val = outputs.eval(feed_dict={X: X_batch})

[[array([[ 0.,  1.,  2.],
       [ 3.,  4.,  5.],
       [ 6.,  7.,  8.],
       [ 9.,  0.,  1.]], dtype=float32), array([[ 9.,  8.,  7.],
       [ 0.,  0.,  0.],
       [ 6.,  5.,  4.],
       [ 3.,  2.,  1.]], dtype=float32)]]


In [ ]:
print(outputs_val)